<a href="https://colab.research.google.com/github/amitadhainje/LearningBERT/blob/master/MukeshSir_Module_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Module-2-Dataset.xlsx to Module-2-Dataset.xlsx


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from bs4 import BeautifulSoup
from sklearn.utils import shuffle
from prettytable import PrettyTable,ALL
import nltk
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import backend as K
import tensorflow_hub as hub
import tensorflow as tf
from textwrap import TextWrapper
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
wrapper = TextWrapper(width=55)
%matplotlib inline

print ("Tensorflow version ===", tf.__version__)
print ("Tensorflow hub version ===", hub.__version__)

Tensorflow version === 2.12.0
Tensorflow hub version === 0.13.0


In [3]:
df = pd.read_excel("Module-2-Dataset.xlsx")
df.head()

,Sentiment,Text
0,shame,When I did not speak the truth.
1,shame,When I could not remember what to say about a ...
2,shame,When I realized that I was directing the feeli...
3,shame,"When I was tired and unmotivated, I shouted at..."
4,shame,"When one has been unjust, stupid towards someo..."


In [4]:
def clean_input_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z.]+', " ", text)
    text = re.sub(r'gt', " ", text)
    contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in text.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)
    text = ' '.join(q_decontracted)

    return text

In [5]:
df['cleaned_text'] = df['Text'].apply(lambda x: clean_input_text(x))
df['max_length'] = df['Text'].apply(lambda x: len(x.split(" ")))
df.head()

,Sentiment,Text,cleaned_text,max_length
0,shame,When I did not speak the truth.,when i did not speak the truth.,7
1,shame,When I could not remember what to say about a ...,when i could not remember what to say about a ...,16
2,shame,When I realized that I was directing the feeli...,when i realized that i was directing the feeli...,34
3,shame,"When I was tired and unmotivated, I shouted at...",when i was tired and unmotivated i shouted at ...,26
4,shame,"When one has been unjust, stupid towards someo...",when one has been unjust stupid towards someon...,10


In [6]:
sentimentCodes = {'shame':0, 'guilt':1, 'disgust':2, 'surprise':3}
df['Sentiment'] = [sentimentCodes[i] for i in df['Sentiment']]
df.head()

,Sentiment,Text,cleaned_text,max_length
0,0,When I did not speak the truth.,when i did not speak the truth.,7
1,0,When I could not remember what to say about a ...,when i could not remember what to say about a ...,16
2,0,When I realized that I was directing the feeli...,when i realized that i was directing the feeli...,34
3,0,"When I was tired and unmotivated, I shouted at...",when i was tired and unmotivated i shouted at ...,26
4,0,"When one has been unjust, stupid towards someo...",when one has been unjust stupid towards someon...,10


In [23]:
# Loading Dependencies
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
# from kaggle_datasets import KaggleDatasets
import transformers

from tokenizers import BertWordPieceTokenizer

In [26]:
# !pip install transformers
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [44]:
import tqdm
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=187):
    """
    Encoder for encoding the text into sequence of integers for BERT Input
    """
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(length=maxlen)
    all_ids = []

    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])

    return np.array(all_ids)

In [39]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = max(df['max_length'])
print (MAX_LEN)

187


In [40]:
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

Tokenizer(vocabulary_size=119547, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=False, wordpieces_prefix=##)

In [41]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

X = df['cleaned_text'].values
y = df['Sentiment'].values
label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(y)
n_classes = 4
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.25, random_state=42)

print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

(14074,)
(4692,)
(14074, 4)
(4692, 4)


In [45]:
x_train = fast_encode(X_train, fast_tokenizer)
x_valid = fast_encode(X_test, fast_tokenizer)

y_train = y_train
y_valid = y_test

TypeError: ignored

In [43]:
pip install